In [2]:
import pandas as pd
import pathlib2 as pathlib

In [16]:
cwd=pathlib.Path.cwd()
datapath=cwd.joinpath('data')
datawreviewsfile=datapath.joinpath('processed/innerJoinData.csv')
dataworeviewsfile=datapath.joinpath('raw/collaborative_book_metadata_with_genredummies.csv')

In [20]:
datadf=pd.read_csv(datawreviewsfile,sep=';')
fulldatadf=pd.read_csv(dataworeviewsfile,sep=';')

In [23]:
# datadf.head()

In [22]:
# fulldatadf.head()

## group books via genres
- vectorize genre-list, countvectorizer vs tfidf (pos: fiction genre very prevalent, thus less relevant in distinction of books)
    - cluster genres (uneven size clusters? not very distinct? fuzzy clustering instead?)
    - topic modeling (lda)

In [2]:
from optuna import Trial
# import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer


### genres
pipeline: tfidf, lda

In [ ]:
#pipe = Pipeline([('tfidf',TfidfVectorizer()),('lda',LatentDirichletAllocation())])